In [1]:
# Inferential Statistics for CT Pretrial Detainees
# (Springboard Capstone 1)
# 2019, Misty M. Giles

# Import everything.  
%matplotlib inline
from datetime import datetime as date
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import scipy.stats as stats
import seaborn as sns

# Select the file created in CT_csv (unit 5 data wrangling assignment)
file = 'detainees_classed_offenses.csv'

In [2]:
# Read in the file
df = pd.read_csv(file, parse_dates=['download_date', 'latest_admission_date'])

# This corrects a typo that's being fixed in CT_csv.
df['offense_class'] = [offense[-2:] if offense.endswith(('AM','BM','CM','DM','UM',' M', 
                       'AF','BF','CF','DF','UF',' F')) else np.nan for offense in df.offense]

# Days that CT says a detainee has been in the system.  This doesn't account for
# some detainees.  There are 730 days in the dataset, and the state says that entrance 
# dates over a year before could be original entrance date but are definitely not to be
# trusted.  The days column will be capped at 1,095 (730 + 365).  This is definitely going
#  to need further investigation.
df.days = [int(time[:-23]) for time in df.days]
df.days = [time if time <= 1095 else 1095 for time in df.days]

# Create age ranges of decades.  For some reason, decades can't be used as a column header.
df['tens'] = [int((age // 10) * 10) for age in df.age] 

# Check that everything worked.  In this EDA, there should be 0 null values.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28808 entries, 0 to 28807
Data columns (total 14 columns):
download_date            28808 non-null datetime64[ns]
identifier               28808 non-null object
latest_admission_date    28808 non-null datetime64[ns]
race                     28808 non-null object
gender                   28808 non-null object
age                      28808 non-null int64
bond_amount              28808 non-null int64
offense                  28808 non-null object
facility                 28808 non-null object
detainer                 28808 non-null object
offense_class            28808 non-null object
days                     28808 non-null int64
counts                   28808 non-null int64
tens                     28808 non-null int64
dtypes: datetime64[ns](2), int64(5), object(7)
memory usage: 3.1+ MB


In [3]:
# Separate out the two datasets, misdemeanor (primary) and felony.
felony_df = df.loc[df['offense_class'].str.endswith('F')]
misdemeanor_df = df.loc[df['offense_class'].str.endswith('M')]

# Divide the misdemeanor data into pre-enactment and post.
date_pre = pd.date_range(start='7/1/2016', end='6/30/2017', freq='D').tolist()
date_post = pd.date_range(start='7/1/2017', end='6/30/2018', freq='D').tolist()
date_end = pd.date_range(start='7/1/2018', end='11/30/2018', freq='D').tolist()
mis_pre_df = misdemeanor_df.loc[misdemeanor_df.download_date < '2017-07-01']
mis_post_df = misdemeanor_df.loc[misdemeanor_df.download_date >= '2017-07-01']
mis_post_df = mis_post_df.loc[mis_post_df.download_date < '2018-07-01']
mis_outofbounds_df = misdemeanor_df.loc[misdemeanor_df.download_date >= '2018-07-01']
mis_df = pd.concat([mis_pre_df, mis_post_df])

# Bin edges to more easily see age differences.
bin_edges = [10, 20, 30, 40, 50, 60, 70, 80, 90] 

# Bin edges for non-age plots.  Sqrt to avoid 'bin bias.'
bins_sqrt = int(np.sqrt(len(misdemeanor_df.bond_amount))) 

# Set up an ecdf function to quickly see distributions.
def ecdf(data):
    '''Compute continuous distribution function for one column'''
    n = len(data)
    x = np.sort(data)
    y = np.arange(1, n+1) / n
    return x, y

# Set up a function to generate bootstrap replicates
def bs_reps(data, func, size=1):
    '''Draw boostrap replicates'''
    # Initialize empty array
    bs_replicates = np.empty(size)
    # Generate replicates
    for i in range(size):
        bs_replicates[i] = func(np.random.choice(data, size=len(data)))
    return bs_replicates

# Set up a function to permute data sets
def permutation_sample(data1, data2):
    '''Generate a permuted sample from two data sets'''
    # Concatenate the data sets
    data = np.concatenate((data1, data2))
    # Permute the concatenated array
    permuted_data = np.random.permutation(data)
    # Split the permuted data back into two data sets
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]
    return perm_sample_1, perm_sample_2

# Set up a function to generate permutation replicates
def draw_perm_reps(data1, data2, func, size=1):
    '''Draw permutation replicates'''
    # Initialize empty array of proper size
    perm_replicates = np.empty(size)
    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data1, data2)
        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)
    return perm_replicates

# Set up a function to calculate differences of means
def diff_of_means(data1, data2):
    '''Calculate difference of means of two arrays'''
    diff = np.mean(data1) - np.mean(data2)
    return diff

def num_means(column):
    '''Get the statistics, mean, and CI for a column'''
    # Get statsitics for column
    m = np.mean(column) # mean of sample
    n = len(column) # sample size
    s = np.std(column) # stdev of sample
    sem = s / np.sqrt(n) # sample error
    
    # Get several bootstrap replicates for column
    bs_replicates = bs_reps(column, np.mean, 10000)
    
    # Get confidence interval
    bootstrap_ci = np.percentile(bs_replicates, [2.5, 97.5])
    
    return bs_replicates, bootstrap_ci

def ptest(column1, column2, test_col):
    '''Get p-value for bootstraps'''
    reps1, ci1 = num_means(column1)
    reps2, ci2 = num_means(column2)
    
    #pval_bs = np.sum((reps1 >= np.mean(test_col)) + (reps2 >= np.mean(test_col))) / (len(reps1) + len(reps2))
    pval_bs1 = np.sum((reps1 >= np.mean(test_col))) / len(reps1)
    pval_bs2 = np.sum((reps2 >= np.mean(test_col))) / len(reps2)
    
    # Print the test results and stats
    print('Bootstrap p-value 1 = ', '%.4f' % pval_bs1)
    print('Bootstrap p-value 2 = ', '%.4f' % pval_bs2)
    print('Mean of observed data     = ', np.mean(test_col))
    print('Bootstrap mean of column1 = ', np.mean(reps1))
    print('Bootstrap mean of column2 = ', np.mean(reps2))
    print('Bootstrap CI column1:\t', ci1[0], ',', ci1[1])
    print('Bootstrap CI column2:\t', ci2[0], ',', ci2[1])

# And for reference, the number of bins in non-age plots:
bins_sqrt

# Seed
np.random.seed(42)

In [4]:
mis_pre_df.head()

,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer,offense_class,days,counts,tens
3,2016-10-11,ZZEBBBSZ,2016-10-07,BLACK,F,57,2500,THREATENING AM,YORK CI,NONE,AM,4,3,50
13,2016-07-13,ZZEBBEWZ,2016-06-17,WHITE,M,63,5000,"ASSAULT, THIRD DEGREE AM",CORRIGAN CI,NONE,AM,26,13,60
14,2017-04-17,ZZEBBHER,2017-01-30,WHITE,M,50,19050,"CRIMINAL TRESPASS, FIRST DEGREE AM",HARTFORD CC,NONE,AM,77,77,50
15,2016-12-06,ZZEBBHER,2016-11-09,WHITE,M,50,500,"CRIMINAL TRESPASS, FIRST DEGREE AM",HARTFORD CC,NONE,AM,27,18,50
16,2017-01-05,ZZEBBHER,2016-12-15,WHITE,M,50,10250,"CRIMINAL TRESPASS, FIRST DEGREE AM",HARTFORD CC,NONE,AM,21,21,50


In [5]:
mis_post_df.head()

,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer,offense_class,days,counts,tens
2,2017-07-13,ZZEBBBJW,2017-07-06,WHITE,M,44,100000,CRIM VIOL OF PROTECTIVE ORDER AM,HARTFORD CC,NONE,AM,7,7,40
4,2017-09-13,ZZEBBBZJ,2017-09-12,BLACK,M,43,20000,THREATENING AM,BRIDGEPORT CC,NONE,AM,1,1,40
5,2018-05-09,ZZEBBBZJ,2018-04-24,BLACK,M,44,150000,THREATENING AM,BRIDGEPORT CC,NONE,AM,15,15,40
8,2018-01-02,ZZEBBCRZ,2017-12-29,WHITE,M,47,2500,"ASSAULT, THIRD DEGREE AM",NEW HAVEN CC,NONE,AM,4,4,40
9,2018-03-22,ZZEBBCRZ,2018-03-21,HISPANIC,M,47,2500,CRIM VIOL OF PROTECTIVE ORDER AM,NEW HAVEN CC,NONE,AM,1,1,40


In [6]:
mis_outofbounds_df.head()

,download_date,identifier,latest_admission_date,race,gender,age,bond_amount,offense,facility,detainer,offense_class,days,counts,tens
7,2018-11-11,ZZEBBCRW,2018-10-10,WHITE,M,55,20000,"CRIMINAL TRESPASS, FIRST DEGREE AM",BRIDGEPORT CC,NONE,AM,32,32,50
11,2018-07-23,ZZEBBEBS,2018-07-09,HISPANIC,M,46,5000,"FAILURE TO APPEAR, SECOND DEGREE AM",HARTFORD CC,NONE,AM,14,14,40
27,2018-09-24,ZZEBBSBS,2018-08-15,BLACK,M,56,2100,DISORDERLY CONDUCT CM,HARTFORD CC,NONE,CM,40,40,50
28,2018-11-11,ZZEBBSLH,2018-05-14,WHITE,M,56,41000,ILL OP MV WO IGNITION DEVICE CM,CORRIGAN CI,NONE,CM,181,179,50
35,2018-08-16,ZZEBCBSZ,2018-07-18,HISPANIC,M,53,20000,"CRIMINAL TRESPASS, FIRST DEGREE AM",BRIDGEPORT CC,NONE,AM,29,29,50


In [7]:
mis_pre_df.shape, mis_post_df.shape, mis_outofbounds_df.shape, mis_df.shape

((3889, 14), (3821, 14), (1954, 14), (7710, 14))

In [8]:
misdemeanor_df.shape

(9664, 14)

In [9]:
mis_pre_df.describe()

,age,bond_amount,days,counts,tens
count,3889.000000,3.889000e+03,3889.000000,3889.000000,3889.000000
mean,36.168167,2.147843e+04,59.048084,27.860375,31.766521
std,11.362481,4.521767e+04,154.314135,35.266749,11.588850
min,18.000000,1.000000e+00,1.000000,1.000000,10.000000
25%,27.000000,2.500000e+03,4.000000,3.000000,20.000000
50%,34.000000,7.500000e+03,22.000000,16.000000,30.000000
75%,45.000000,2.500000e+04,54.000000,38.000000,40.000000
max,75.000000,1.025000e+06,1095.000000,287.000000,70.000000


Okay, now that everything is set up, I'm going to start looking for the variables of interest: gender, race, age, offense_class, days, and bond_amount.  Are any of these numeric variables normal?

In [10]:
chisq, pval = stats.normaltest(mis_pre_df.age)
if pval < .05:
    print('Normal distribution IS NOT a good fit for age.  (p-value = ', pval, ')')
else:
    print('Normal distribution IS a good fit for age.  (p-value = ', pval, ')')
    
chisq, pval = stats.normaltest(mis_pre_df.days)
if pval < .05:
    print('Normal distribution IS NOT a good fit for days.  (p-value = ', pval, ')')
else:
    print('Normal distribution IS a good fit for days.  (p-value = ', pval, ')')
    
chisq, pval = stats.normaltest(mis_pre_df.bond_amount)
if pval < .05:
    print('Normal distribution IS NOT a good fit for bond.  (p-value = ', pval, ')')
else:
    print('Normal distribution IS a good fit for bond.  (p-value = ', pval, ')')

Normal distribution IS NOT a good fit for age.  (p-value =  2.557558769554361e-57 )
Normal distribution IS NOT a good fit for days.  (p-value =  0.0 )
Normal distribution IS NOT a good fit for bond.  (p-value =  0.0 )


##### First set of tests: gender

Below we'll look at the bond and days means and see who is expected to stay longer and pay more.

For both variables:
H0: male == female
Ha: male != female 
alpha = .05

Fascinatingly, the p-values for both means are about .49, a value high enough that we fail to reject H0 for both tests.  The bond amount and days detained for both genders can be considered statistically equal.

In [11]:
# Determine the populations
female = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.gender == 'F']
male = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.gender == 'M']

In [12]:
# Test female vs. male for bond amounts
ptest(female.bond_amount, male.bond_amount, mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0000
Bootstrap p-value 2 =  0.9759
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  13762.90676607143
Bootstrap mean of column2 =  23092.35926677028
Bootstrap CI column1:	 11656.23947172619 , 16276.998474702381
Bootstrap CI column2:	 21500.68378924464 , 24784.367819396954


In [13]:
# Test female vs. male for days detained
ptest(female.days, male.days, mis_pre_df.days)

Bootstrap p-value 1 =  0.0000
Bootstrap p-value 2 =  0.9733
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  32.78871324404762
Bootstrap mean of column2 =  64.50434311470313
Bootstrap CI column1:	 27.065438988095238 , 39.52529761904762
Bootstrap CI column2:	 58.96423686664594 , 70.2545927883121


##### Second set of tests: race

Below we'll again look at the bond and days means and see who is expected to stay longer and pay more.

For both variables: 
* H0: white == black / minority 
* Ha: white != black / minority
* alpha = .05

Interestingly, we refuse to reject the null hypothesis for all of the bond amounts.  For the days detained, however, we would reject the null for Asians and Native Americans.  This sample is too small to come to this conclusion.  (52 Asians+Native Americans out of 3889 detainees, or 1.3% of the sample)

In [14]:
# Determine the populations
race_df = mis_pre_df[['bond_amount', 'days', 'race']].copy()
race_df['r'] = mis_pre_df[['race']].copy()
race_df = pd.get_dummies(race_df, columns=['r'])
race_df_post = mis_post_df[['bond_amount', 'days', 'race']].copy()
race_df_post['r'] = mis_post_df[['race']].copy()
race_df_post = pd.get_dummies(race_df_post, columns=['r'])
white = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.race == 'WHITE']
minority = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.race != 'WHITE']
black = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.race == 'BLACK']
hispanic = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.race == 'HISPANIC']
asian = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.race == 'ASIAN']
amer_ind = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.race == 'AMER IND']
white_post = mis_post_df[['bond_amount', 'days']].loc[mis_post_df.race == 'WHITE']
minority_post = mis_post_df[['bond_amount', 'days']].loc[mis_post_df.race != 'WHITE']
black_post = mis_post_df[['bond_amount', 'days']].loc[mis_post_df.race == 'BLACK']
hispanic_post = mis_post_df[['bond_amount', 'days']].loc[mis_post_df.race == 'HISPANIC']
asian_post = mis_post_df[['bond_amount', 'days']].loc[mis_post_df.race == 'ASIAN']
amer_ind_post = mis_post_df[['bond_amount', 'days']].loc[mis_post_df.race == 'AMER IND']

In [15]:
# Test white vs. non-white for bond amount
ptest(white.bond_amount, minority.bond_amount, mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0004
Bootstrap p-value 2 =  0.9857
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  18374.43814360856
Bootstrap mean of column2 =  23727.593361180123
Bootstrap CI column1:	 16874.362048929663 , 19995.959678899082
Bootstrap CI column2:	 21669.06462954747 , 25996.33307453416


In [16]:
# Test white vs. black for bond amount
ptest(white.bond_amount, black.bond_amount, mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0003
Bootstrap p-value 2 =  0.8363
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  18377.590293272173
Bootstrap mean of column2 =  22832.748686588235
Bootstrap CI column1:	 16816.43778287462 , 20030.59110091743
Bootstrap CI column2:	 20285.76745098039 , 25627.996117647057


In [17]:
# Test white vs. hispanic for bond amount
ptest(white.bond_amount, hispanic.bond_amount, mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0001
Bootstrap p-value 2 =  0.9894
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  18355.375782018345
Bootstrap mean of column2 =  25370.432859115426
Bootstrap CI column1:	 16810.526911314984 , 19978.124250764522
Bootstrap CI column2:	 21967.047491909383 , 29277.281499460627


In [18]:
# Test white vs. Asian for bond amount
ptest(white.bond_amount, asian.bond_amount, mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0001
Bootstrap p-value 2 =  0.1651
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  18373.99280281346
Bootstrap mean of column2 =  15800.018457142856
Bootstrap CI column1:	 16777.859801223243 , 20002.29510703364
Bootstrap CI column2:	 6994.304285714286 , 28957.449285714283


In [19]:
# Test white vs. Native American for bond amount
ptest(white.bond_amount, amer_ind.bond_amount, mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0000
Bootstrap p-value 2 =  0.2926
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  18375.66690434251
Bootstrap mean of column2 =  18882.131470588236
Bootstrap CI column1:	 16848.421269113154 , 20007.73085626911
Bootstrap CI column2:	 9852.573529411764 , 29338.308823529413


In [20]:
# Test white vs. non-white for days detained
ptest(white.days, minority.days, mis_pre_df.days)

Bootstrap p-value 1 =  0.0094
Bootstrap p-value 2 =  0.9414
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  51.33946042813455
Bootstrap mean of column2 =  64.62605550133095
Bootstrap CI column1:	 45.38275229357798 , 57.77081039755351
Bootstrap CI column2:	 57.66500665483585 , 71.88664596273291


In [21]:
# Test white vs. black for days detained
ptest(white.days, black.days, mis_pre_df.days)

Bootstrap p-value 1 =  0.0112
Bootstrap p-value 2 =  0.9882
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  51.384512048929665
Bootstrap mean of column2 =  70.30228690196078
Bootstrap CI column1:	 45.40266055045871 , 57.79093272171254
Bootstrap CI column2:	 60.5293137254902 , 80.98123529411765


In [22]:
# Test white vs. hispanic for days detained
ptest(white.days, hispanic.days, mis_pre_df.days)

Bootstrap p-value 1 =  0.0104
Bootstrap p-value 2 =  0.4176
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  51.349767461773695
Bootstrap mean of column2 =  58.1319405609493
Bootstrap CI column1:	 45.33923547400612 , 57.76964831804281
Bootstrap CI column2:	 48.854314994606256 , 68.51504854368932


In [23]:
# Test white vs. Asian for days detained
ptest(white.days, asian.days, mis_pre_df.days)

Bootstrap p-value 1 =  0.0089
Bootstrap p-value 2 =  0.0490
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  51.340873027522946
Bootstrap mean of column2 =  42.12347428571428
Bootstrap CI column1:	 45.36755351681957 , 57.719877675840976
Bootstrap CI column2:	 25.513571428571428 , 62.200714285714284


In [24]:
# Test white vs. Native American for days detained
ptest(white.days, amer_ind.days, mis_pre_df.days)

Bootstrap p-value 1 =  0.0125
Bootstrap p-value 2 =  0.0297
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  51.355428746177374
Bootstrap mean of column2 =  41.36328235294118
Bootstrap CI column1:	 45.42076452599388 , 57.90492354740061
Bootstrap CI column2:	 24.0 , 59.76617647058821


In [25]:
# Test white pre vs. post for bond
ptest(white.bond_amount, white_post.bond_amount, mis_df.bond_amount)

Bootstrap p-value 1 =  0.0001
Bootstrap p-value 2 =  0.4193
Mean of observed data     =  22212.647600518805
Bootstrap mean of column1 =  18368.501769847095
Bootstrap mean of column2 =  22019.52606676558
Bootstrap CI column1:	 16802.899525993886 , 20046.10340978593
Bootstrap CI column2:	 19738.824792284864 , 24504.128442136498


In [26]:
# Test minority pre vs. post for bond
ptest(minority.bond_amount, minority_post.bond_amount, mis_df.bond_amount)

Bootstrap p-value 1 =  0.9232
Bootstrap p-value 2 =  0.9328
Mean of observed data     =  22212.647600518805
Bootstrap mean of column1 =  23745.385731455193
Bootstrap mean of column2 =  23709.717830992508
Bootstrap CI column1:	 21677.137522182788 , 26022.285891748004
Bootstrap CI column2:	 21796.351802434456 , 25775.441690074902


In [27]:
# Test black pre vs. post for bond
ptest(black.bond_amount, black_post.bond_amount, mis_df.bond_amount)

Bootstrap p-value 1 =  0.6553
Bootstrap p-value 2 =  0.7485
Mean of observed data     =  22212.647600518805
Bootstrap mean of column1 =  22808.85969678431
Bootstrap mean of column2 =  23182.3426923708
Bootstrap CI column1:	 20303.01419607843 , 25694.842901960783
Bootstrap CI column2:	 20573.434003281378 , 26093.297662018045


In [28]:
# Test white pre vs. post for days
ptest(white.days, white_post.days, mis_df.days)

Bootstrap p-value 1 =  0.0061
Bootstrap p-value 2 =  0.0005
Mean of observed data     =  59.996887159533074
Bootstrap mean of column1 =  51.345047339449536
Bootstrap mean of column2 =  49.40016783382789
Bootstrap CI column1:	 45.432247706422025 , 57.790474006116206
Bootstrap CI column2:	 43.82839762611276 , 55.35261127596439


In [29]:
# Test minority pre vs. post for days
ptest(minority.days, minority_post.days, mis_df.days)

Bootstrap p-value 1 =  0.9018
Bootstrap p-value 2 =  0.9941
Mean of observed data     =  59.996887159533074
Bootstrap mean of column1 =  64.64230292812776
Bootstrap mean of column2 =  69.98020964419476
Bootstrap CI column1:	 57.830301685891754 , 72.0213065661047
Bootstrap CI column2:	 62.126135299625474 , 78.35173220973783


In [30]:
# Test black pre vs. post for days
ptest(black.days, black_post.days, mis_df.days)

Bootstrap p-value 1 =  0.9778
Bootstrap p-value 2 =  0.9986
Mean of observed data     =  59.996887159533074
Bootstrap mean of column1 =  70.29170564705882
Bootstrap mean of column2 =  76.14637834290403
Bootstrap CI column1:	 60.205411764705886 , 80.86388235294116
Bootstrap CI column2:	 64.97696882690731 , 87.90812141099262


##### Third set of tests: offense class

Below we'll again look at the bond and days means and see if the class of offense affects bond amount or days detained.

For both variables: 


In [31]:
offense_df = mis_pre_df[['bond_amount', 'days', 'offense_class']].copy()
offense_df['class'] = mis_pre_df[['offense_class']].copy()
offense_df = pd.get_dummies(offense_df, columns=['class'])
offense_am = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.offense_class == 'AM']
offense_bm = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.offense_class == 'BM']
offense_cm = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.offense_class == 'CM']
offense_dm = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.offense_class == 'DM']
offense_um = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.offense_class == 'UM']
offense_m  = mis_pre_df[['bond_amount', 'days']].loc[mis_pre_df.offense_class == ' M']

In [32]:
# Test A-class vs. all others for bond amount
ptest(offense_am.bond_amount, mis_pre_df.bond_amount.loc[mis_pre_df.offense_class != 'AM'], mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.9954
Bootstrap p-value 2 =  0.0000
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  23631.158598063626
Bootstrap mean of column2 =  15265.67269538616
Bootstrap CI column1:	 21966.67291666667 , 25365.89268672199
Bootstrap CI column2:	 13100.075677031093 , 17740.71925777332


In [33]:
# Test B-class vs. all others for bond amount
ptest(offense_bm.bond_amount, mis_pre_df.bond_amount.loc[mis_pre_df.offense_class != 'BM'], mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0027
Bootstrap p-value 2 =  0.7151
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  16486.645613826364
Bootstrap mean of column2 =  21930.683719591947
Bootstrap CI column1:	 13407.062942122187 , 19940.308440514465
Bootstrap CI column2:	 20471.208873672444 , 23492.725845444384


In [34]:
# Test C-class vs. all others for bond amount
ptest(offense_cm.bond_amount, mis_pre_df.bond_amount.loc[mis_pre_df.offense_class != 'CM'], mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.0000
Bootstrap p-value 2 =  0.9173
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  13083.361817582418
Bootstrap mean of column2 =  22582.14829618521
Bootstrap CI column1:	 11039.622472527473 , 15260.589835164834
Bootstrap CI column2:	 21035.29804892254 , 24239.03545428072


In [35]:
# Test no-class vs. all others for bond amount
ptest(offense_m.bond_amount, mis_pre_df.bond_amount.loc[mis_pre_df.offense_class != ' M'], mis_pre_df.bond_amount)

Bootstrap p-value 1 =  0.1914
Bootstrap p-value 2 =  0.6141
Mean of observed data     =  21478.429673437902
Bootstrap mean of column1 =  18007.22699521739
Bootstrap mean of column2 =  21708.696636622026
Bootstrap CI column1:	 11040.81108695652 , 26879.714021739128
Bootstrap CI column2:	 20344.24259360481 , 23196.347936594695


No point in testing UM.  There's only one.

In [36]:
# Test A-class vs. all others for days detained
ptest(offense_am.days, mis_pre_df.days.loc[mis_pre_df.offense_class != 'AM'], mis_pre_df.days)

Bootstrap p-value 1 =  0.1954
Bootstrap p-value 2 =  0.8842
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  56.75000491009682
Bootstrap mean of column2 =  65.81520280842528
Bootstrap CI column1:	 51.59140733056709 , 62.16230117565698
Bootstrap CI column2:	 54.9537111334002 , 77.40744734202607


In [37]:
# Test B-class vs. all others for days detained
ptest(offense_bm.days, mis_pre_df.days.loc[mis_pre_df.offense_class != 'BM'], mis_pre_df.days)

Bootstrap p-value 1 =  0.8330
Bootstrap p-value 2 =  0.3610
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  69.12334790996785
Bootstrap mean of column2 =  58.167014784795974
Bootstrap CI column1:	 50.073874598070745 , 91.30627009646301
Bootstrap CI column2:	 53.40932783678032 , 63.24686975964225


In [38]:
# Test C-class vs. all others for days detained
ptest(offense_cm.days, mis_pre_df.days.loc[mis_pre_df.offense_class != 'CM'], mis_pre_df.days)

Bootstrap p-value 1 =  0.9813
Bootstrap p-value 2 =  0.1588
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  77.70379802197802
Bootstrap mean of column2 =  56.59922979033198
Bootstrap CI column1:	 60.151593406593406 , 97.10373626373627
Bootstrap CI column2:	 51.75684333139196 , 61.66231799650553


In [39]:
# Test no-class vs. all others for days detained
ptest(offense_m.days, mis_pre_df.days.loc[mis_pre_df.offense_class != ' M'], mis_pre_df.days)

Bootstrap p-value 1 =  0.0144
Bootstrap p-value 2 =  0.6903
Mean of observed data     =  59.04808434044742
Bootstrap mean of column1 =  37.988419130434785
Bootstrap mean of column2 =  60.36759937141296
Bootstrap CI column1:	 23.35630434782609 , 56.360978260869565
Bootstrap CI column2:	 55.514867450122985 , 65.56311150587592


In [40]:
def prop_test(col1, col2, orig_col, tail=2, percent=.975):
    '''I have no clue what this does or why it is here, tbh.  Wrote it thinking it would do something different.'''
    n1 = len(col1)
    n2 = len(col2)
    n3 = len(orig_col)
    
    pro1 = np.mean(col1)
    pro2 = np.mean(col2)
    pro3 = (np.sum(col1)+np.sum(col2))/n3
    
    p_1 = pro1*(1-pro1)
    p_2 = pro2*(1-pro2)
    p_3 = pro3*(1-pro3)
    
    observed_diff = diff_of_means(col1, col2)
    
    z = observed_diff / np.sqrt(p_3*((1/n1)+(1/n2)))
    z_crit = stats.norm.ppf(percent)
    moe = z_crit*np.sqrt((p_1/n1)+(p_2/n2))
    ci_high = observed_diff + moe
    ci_low = observed_diff - moe
    z_pval = (1-stats.norm.cdf(z))*tail
    
    print('The margin of error is', '{:.7f}'.format(moe), '(', '{:.2f}'.format(moe*100), 'percent )')
    print('The 95% confidence interval is', '{:.7f}'.format(ci_low), 'to', '{:.7f}'.format(ci_high), '(','{:.2f}'.format(ci_low*100), 'to', '{:.2f}'.format(ci_high*100), 'percent difference )')
    print('The p-value is', z_pval, '(', '{:.7f}'.format(z_pval), ')')

In [41]:
offense_df.describe()

,bond_amount,days,class_ M,class_AM,class_BM,class_CM,class_UM
count,3.889000e+03,3889.000000,3889.000000,3889.000000,3889.000000,3889.000000,3889.000000
mean,2.147843e+04,59.048084,0.059141,0.743636,0.079969,0.116997,0.000257
std,4.521767e+04,154.314135,0.235919,0.436681,0.271280,0.321458,0.016035
min,1.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.500000e+03,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.500000e+03,22.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,2.500000e+04,54.000000,0.000000,1.000000,0.000000,0.000000,0.000000
max,1.025000e+06,1095.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [42]:
race_df.describe()

,bond_amount,days,r_AMER IND,r_ASIAN,r_BLACK,r_HISPANIC,r_WHITE
count,3.889000e+03,3889.000000,3889.000000,3889.000000,3889.000000,3889.000000,3889.000000
mean,2.147843e+04,59.048084,0.004371,0.009000,0.327848,0.238365,0.420417
std,4.521767e+04,154.314135,0.065980,0.094451,0.469489,0.426138,0.493689
min,1.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.500000e+03,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,7.500000e+03,22.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.500000e+04,54.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.025000e+06,1095.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [43]:
race = 

SyntaxError: invalid syntax (<ipython-input-43-a3b964d36604>, line 1)

In [44]:
prop_test(race_df.r_BLACK, race_df.r_HISPANIC, race_df.race)

The margin of error is 0.0199248 ( 1.99 percent )
The 95% confidence interval is 0.0695584 to 0.1094080 ( 6.96 to 10.94 percent difference )
The p-value is 1.7763568394002505e-15 ( 0.0000000 )


In [45]:
test = pd.concat([race_df.r_WHITE, race_df_post.r_WHITE])
prop_test(race_df.r_WHITE, race_df_post.r_WHITE, test)

The margin of error is 0.0221026 ( 2.21 percent )
The 95% confidence interval is -0.0426701 to 0.0015351 ( -4.27 to 0.15 percent difference )
The p-value is 1.931777773438304 ( 1.9317778 )


In [46]:
race_df_post.describe()

,bond_amount,days,r_AMER IND,r_ASIAN,r_BLACK,r_HISPANIC,r_WHITE
count,3.821000e+03,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000,3821.000000
mean,2.295993e+04,60.962575,0.003141,0.005234,0.319026,0.231615,0.440984
std,4.813973e+04,164.002986,0.055960,0.072168,0.466160,0.421919,0.496570
min,1.000000e+00,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.500000e+03,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000e+04,15.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.500000e+04,53.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,1.000150e+06,1095.000000,1.000000,1.000000,1.000000,1.000000,1.000000


n_b = len(b.call)
n_w = len(w.call)
cb = np.mean(data.call)
callbacks_w = np.mean(w.call)
callbacks_b = np.mean(b.call)

cb_p = cb*(1-cb)
cb_pb = callbacks_b*(1-callbacks_b)
cb_pw = callbacks_w*(1-callbacks_w)

tail = 2

z = observed_diff / np.sqrt(cb_p*((1/n_b)+(1/n_w)))
z_crit = stats.norm.ppf(.975)
moe = z_crit*np.sqrt((cb_pb/n_b)+(cb_pw/n_w))
ci_high = observed_diff + moe
ci_low = observed_diff - moe
z_pval = (1-stats.norm.cdf(z))*tail

print('The margin of error is', '{:.7f}'.format(moe), '(', '{:.2f}'.format(moe*100), 'percent )')
print('The 95% confidence interval is', '{:.7f}'.format(ci_low), 'to', '{:.7f}'.format(ci_high), '(','{:.2f}'.format(ci_low*100), 'to', '{:.2f}'.format(ci_high*100), 'percent difference )')
print('The p-value is', z_pval, '(', '{:.7f}'.format(z_pval), ')')